# Advanced Indexing

We conclude our discussion of indexing into N-dimensional NumPy arrays by understanding advanced indexing. Unlike basic indexing, which allows us to access distinct elements and regular slices of an array, advanced indexing is significantly more flexible. For example, it permits the use of *boolean-valued* arrays as indices, 

```python
# Use a boolean-valued array to access
# the diagonal values of an array
>>> import numpy as np

>>> x = np.array([[0, 1, 2],
...               [3, 4, 5],
...               [6, 7, 8]])

# Specify `True` wherever we want to access 
# the the entry of `x`
>>> bool_index = np.array([[ True, False, False],
...                        [False,  True, False],
...                        [False, False,  True]])

>>> x[bool_index]
array([0, 4, 8])
```

Additionally, arrays of integers can be used to access arbitrary and even repeated entries from an array,

```python
# Construct the following 2D array
# from the contents of `x`:
#
#     [[x[0, 0], x[0, 1]],
#      [x[2, 2], x[2, 2]]]
>>> rows = np.array([[0, 0], 
...                  [2, 2]])

>>> cols = np.array([[0, 1], 
...                  [2, 2]])

>>> x[rows, cols]
array([[0, 1],
       [8, 8]])
```

Unlike basic indexing, advanced indexing always produces a copy of the underlying data.
```python
>>> np.shares_memory(x, x[bool_index])
False

>>> np.shares_memory(x, x[rows, cols])
False
```
The flexibility permitted by advanced indexing makes it a difficult topic to treat exhaustively without delving into somewhat terse and abstract notation. It is best to refer to [the official documentation](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing) for such a treatment of the topic. Here, we will discuss the essential aspects of advanced indexing, with the aim of the discussion being  both thorough and accessible.

<div class="alert alert-info"> 

**Definition: Advanced Indexing**: 

Given an $N$-dimensional array, `x`, `x[index]` invokes **advanced indexing** whenever `index` is:

- an integer-type or boolean-type `numpy.ndarray`
- a `tuple` with at least one *sequence*-type object as an element

Accessing the contents of an array via advanced indexing *always returns a copy of those contents*, whereas basic indexing returns a view.

</div>

## Integer Array Indexing

### Indexing into 1-Dimensional Arrays
Using an integer-type array as an index allows us to access the contents of an array arbitrarily, permitting items to be accessed out of order, and even repeatedly. Consider the following 1-dimensional array.

```python
y = np.array([ 0, -1, -2, -3, -4, -5])
```

See that we can access its contents in an unpatterned way, which is not permissible via basic index:

```python
# advanced indexing with an integer-array
>>> index = np.array([2, 4, 0, 4, 4, 4])
>>> y[index]
array([-2, -4,  0, -4, -4, -4])
```


The specification for accessing the contents of `y` in this way is straight-forward to interpret. Each entry of the index-array is used to access an element from `y`, as illustrated here:

\begin{equation}
\left(
\begin{array}{*{6}{X}}
  y[2] & y[4] & y[0] & y[4] & y[4] & y[4]
\end{array}
\right)
% 
\rightarrow
\left(
\begin{array}{*{6}{X}}
  -2 & -4 & 0 & -4 & -4 & -4
\end{array}
\right)
\end{equation}

This returns a *copy* of the data, as do all occurrences of advanced indexing.

```python
# advanced indexing returns a copy
>>> np.shares_memory(y, y[index])
```

The indexing array can have an arbitrary shape; the resulting array will match that shape.

```python
# utilizing a 2D-array as an index
>>> index_2d = np.array([[ 1,  2,  0],
...                      [ 5,  5,  5],
...                      [ 2,  3,  4]])

# the resulting shape matches the shape of the indexing array
>>> y[index_2d]
array([[-1, -2,  0],
       [-5, -5, -5],
       [-2, -3, -4]])
```

\begin{equation}
\left(
\begin{array}{*{3}{X}}
  y[1] & y[2] & y[0] \\
  y[5] & y[5] & y[5] \\
  y[2] & y[3] & y[4]
\end{array}
\right)
% 
\rightarrow
\left(
\begin{array}{*{3}{X}}
  -1 & -2 & 0 \\
  -5 & -5 & -5 \\
  -2 & -3 & -4
\end{array}
\right)
\end{equation}

<div class="alert alert-info"> 

**Reading Comprehension: Array Indexing (1-D)**

Given the following array:

```python
y = np.array([ 0, -1, -2, -3, -4, -5])
```

Use advanced indexing, using an integer-array, to produce the following arrays:

```python
# 1
array([-1])

#2
array([-1, -2, -1, -2])

#3
array([[ 0, -5],
       [-1, -4]])

#4
array([[-2],
       [-3],
       [-2]])
```

</div>

### Indexing into N-Dimensional Arrays

In the preceding examples, we specified a single index-array array to access the contents along the only dimension of a flat array. As you may guess, in order to perform this variety of indexing on an $N$-dimensional array, we must specify $N$ index-arrays. Each of the $N$ index-arrays must have the same shape. The common shape of the index-arrays determines the shape of the resulting array. 

The corresponding entries of each of the $N$ index arrays is used to specify a specific array element to be accessed. For example, consider the following 3-dimensional array whose elements we will be accessing:

```python
# Indexing a 3D array using integer index-arrays
>>> z = np.array([[[ 0,  1,  2,  3],
...                [ 4,  5,  6,  7],
...                [ 8,  9, 10, 11]],
...
...               [[12, 13, 14, 15],
...                [16, 17, 18, 19],
...                [20, 21, 22, 23]]])
```

We specify three index-arrays, the indices to be accessed along axis-0, axis-1, and axis-2, respectively. Suppose we want to produce the array: `array([ 3, 23,  4])`. The layout of these elements is as follows:

- `3`: sheet-0, row-0, column-3
- `23`: sheet-1, row-2, column-3
- `4`: sheet-0, row-1, column-0

Each index-array must a shape (3,) in order to produce the result of the appropriate shape. The index-array supplied for axis-0 must be `np.array([0, 1, 0])` in order to select sheet-0 for `3`, sheet-1 for `23`, and then sheet-0 for `4`. The other index arrays are formed accordingly. 

```python
# specifies subsequent sheets to access
>>> index_0 = np.array([0, 1, 0])

# specifies subsequent rows to access
>>> index_1 = np.array([0, 2, 1])

# specifies subsequent columns to access
>>> index_2 = np.array([3, 3, 0])

>>> z[index_0, index_1, index_2]
array([ 3, 23,  4])
```
\begin{equation}
\left(
\begin{array}{*{3}{X}}
   z[0, 0, 3] & z[1, 2, 3] & z[0, 1, 0] 
\end{array}
\right)
% 
\rightarrow
\left(
\begin{array}{*{3}{X}}
  3 & 23 & 4
\end{array}
\right)
\end{equation}

The index arrays are traversed using [row-major ordering](https://www.pythonlikeyoumeanit.com/Module3_IntroducingNumpy/ArrayTraversal.html), when populating the resulting array.

<div class="alert alert-info"> 

**Reading Comprehension: Array Indexing (N-D)**

Given the following array:

```python
>>> z = np.array([[[ 0,  1,  2,  3],
...                [ 4,  5,  6,  7],
...                [ 8,  9, 10, 11]],
...
...               [[12, 13, 14, 15],
...                [16, 17, 18, 19],
...                [20, 21, 22, 23]]])
```

Use advanced indexing, using integer-arrays, to produce the following arrays:

```python
# 1
array([[ 0,  5, 10],
       [12, 17, 22]])

#2
array([[ 0, 23],
       [23,  0]])

```

</div>

<div class="alert alert-info"> 

**Takeaway:**

An $N$-dimensional array's contents can be accessed by supplying $N$ index-arrays of integers, one for each axis of data. The index-arrays must have the same shape as one another, and this common shape determines the shape of the resulting array. This is a form of advanced indexing, and thus a copy of the parent array's data is created.

</div>

## Boolean Array Indexing
NumPy also permits the use of a boolean-value array as an index, to perform advanced indexing on an array. In its simplest form, this is an extremely intuitive and elegant method for selecting contents from an array based on logical conditions.

```python
# advanced indexing using a boolean-array
>>> x = np.array([[[-0.26,  0.49,  0.18],
...                [ 0.43,  0.3 ,  0.29]],
...        
...               [[-0.44,  0.3 ,  0.28],
...                [ 0.27, -0.09, -0.13]]])

# `True` wherever `x` is positive
>>> bool_ind = x > 0
>>> bool_ind
array([[[False,  True,  True],
        [ True,  True,  True]],

       [[False,  True,  True],
        [ True, False, False]]], dtype=bool)

>>> x[bool_ind]
array([ 0.49,  0.18,  0.43,  0.3 ,  0.29,  0.3 ,  0.28,  0.27])
```
In its simplest form, boolean indexing behaves as follows: Suppose `x` is an $N$-dimensional array, and `ind` is a boolean-value *of the same shape as* `x`. Then `x[ind]` returns a 1-dimensional array, which is formed by traversing `x` and `ind` using [row-major ordering](https://www.pythonlikeyoumeanit.com/Module3_IntroducingNumpy/ArrayTraversal.html). Wherever an element of `ind` is `True`, the corresponding entry of `x` is added to the end of the resulting array. Refer to the preceding example and convince yourself that this is the behavior that is exhibited.

In [22]:
x[np.array([[True], [True], [True]])]

IndexError: boolean index did not match indexed array along dimension 1; dimension is 3 but corresponding boolean dimension is 1

### Converting from Boolean Arrays to Integer Index-Arrays: numpy.where

NumPy provides 

In [18]:
>>> x = np.array([[0, 1, 2],
...               [3, 4, 5],
...               [6, 7, 8]])

In [16]:
x[rows, cols] *= 33

In [17]:
x

array([[  0,  33,   2],
       [  3,   4,   5],
       [  6,   7, 264]])

In [5]:
index = np.array([[True, False, False], [False, True, False], [False, False, True]])

In [6]:
index

array([[ True, False, False],
       [False,  True, False],
       [False, False,  True]], dtype=bool)

## Links to Official Documentation

- [Advanced Indexing](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing)

## Reading Comprehension Solutions

**Array Indexing (1-D): Solution**

```python
y = np.array([ 0, -1, -2, -3, -4, -5])
```

```python
# 1
>>> ind1 = np.array([1])
>>> y[ind1]
array([-1])

#2
>>> ind2 = np.array([1, 2, 1, 2])
>>> y[ind2]
array([-1, -2, -1, -2])

#3
>>> ind3 = np.array([[0, 5],
...                  [1, 4]])
>>> y[ind3]
array([[ 0, -5],
       [-1, -4]])

#4
>>> ind4 = np.array([[2],
...                  [3],
...                  [2]])
>>> y[ind4]
array([[-2],
       [-3],
       [-2]])
```

**Array Indexing (N-D): Solution**

```python
>>> z = np.array([[[ 0,  1,  2,  3],
...                [ 4,  5,  6,  7],
...                [ 8,  9, 10, 11]],
...
...               [[12, 13, 14, 15],
...                [16, 17, 18, 19],
...                [20, 21, 22, 23]]])
```

```python
# 1
>>> ind0 = np.array([0, 0, 0, 1, 1, 1]).reshape(2, 3)
>>> ind1 = np.array([0, 1, 2, 0, 1, 2]).reshape(2, 3)
>>> ind2 = np.array([0, 1, 2, 0, 1, 2]).reshape(2, 3)
>>> z[ind0, ind1, ind2]
array([[ 0,  5, 10],
       [12, 17, 22]])

# 2
>>> ind0 = np.array([0, 1, 1, 0]).reshape(2, 2)
>>> ind1 = np.array([0, 2, 2, 0]).reshape(2, 2)
>>> ind2 = np.array([0, 3, 3, 0]).reshape(2, 2)
>>> z[ind0, ind1, ind2]
array([[ 0, 23],
       [23,  0]])
```